In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy

# Problema 2:
Considere el entramado mostrado en la figura, con una fuerza aplicada de 20 kN. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen E = 210 GPa y una sección de 10 cm2, excepto el elemento 3, que tiene una sección de 20 cm2. Los elementos 2 y 5 tienen una longitud de 8 metros y el elemento 3 de 4 metros.

In [2]:
def loc_a_glob (z):
    z=z*np.pi/180
    c=np.cos(z)
    s=np.sin(z)
    K=[[ c**2,    c*s ,   -c**2, -c*s] , 
       [ c*s,      s**2,   -c*s, -s**2] , 
       [-c**2,    -c*s,    c**2,  c*s], 
       [-c*s,     -s**2,    c*s,   s**2]]
    return K

In [5]:
N = 4                      # cantidad de nodos = cantidad de elementos -1
E = np.ones(N+1)*210            # Módulo elástico de cada elemento GPa
A = np.ones(N+1)*10             # Area de cada elementos cm2
A[2] = 20
L = np.array([0, 8, 4, 0, 8])*100  # Longitud de cada elemento cm
GL = 2                               # Grados de libertad

MN = [[0,0] , [L[1],L[2]] , [L[1]+L[4],0] , [L[1],0]]  # MAtriz de nodos (posiciones de los nodos) (ejercicio)
        
MC = [ [1, 0] , [3,0] , [1,3] , [2,3], [2, 1]  ]      # Matriz de conectividad de elementos (el  elemento 0: entre nodo 0: MN[0] y nodo 1 MN[1])  ) (ejercicio)


ff=np.array([[2,0], [4,0], [5,-20000], [6,0]]) # columnas: nodo y eje - filas: valor de fuerza conocido

# no hice una matriz de datos desplazamientos pq son todos ceros, cuando el codigo funcione, lo generalizo.

u = np.zeros(N*GL)


K = np.zeros([N*GL,N*GL])     # Matriz rigidez GLOBAL kN/m



for i in range(len(MC)):
    dx= MN[MC[i][0]][0] - MN[MC[i][1]][0]
    dy = MN[MC[i][0]][1] - MN[MC[i][1]][1]
    dL=np.sqrt( dx ** 2 + dy ** 2 )
    k=E[i]*A[i]/dL    
    z=np.arctan2(dy,dx)          # Creo que el error esta acá o en R
    R=k*np.array(loc_a_glob(z))  # Matriz de rigidez local rotada
    n1=MC[i][0]*2                #index nodo 1 en x del elemento i
    n2=MC[i][1]*2                #index nodo 2 en x del elemento i
    K[n1:n1+2,n1:n1+2]+=R[0:2,0:2]
    K[n1:n1+2,n2:n2+2]+=R[0:2,2:4]
    K[n2:n2+2,n1:n1+2]+=R[2:4,0:2]
    K[n2:n2+2,n2:n2+2]+=R[2:4,2:4]

#Condiciones de contorno:

kk=np.zeros((len(ff),len(ff)))   #Matriz reducida para calcular desplazamientos desconocidos
for i in range(len(ff)):
    for j in range(len(ff)):
        kk[i, j]=K[ff[i,0], ff[j,0]]
uu=np.linalg.solve(kk, ff[:,1])

for i in range(len(ff)):
    u[ff[i,0]]=uu[i]

F=np.linalg.solve(K,u)

In [6]:
F, u

(array([ 9.82806640e+24, -4.49060764e+27, -1.53899048e+25, -1.37433624e+27,
         9.82806640e+24,  1.74193515e+27,  9.82806640e+24, -2.29394707e+27]),
 array([ 0.00000000e+00,  0.00000000e+00,  1.11208990e+05,  0.00000000e+00,
        -1.04097374e+06, -2.72466328e+08, -9.94618315e+04,  0.00000000e+00]))

# Confirmación de suma de rigidez local en la global:

x=np.array([[1,1,1,1], [2,2,2,2],[3,3,3,3],[4,4,4,4]])
y=np.array([[5,5,5,5], [6,6,6,6],[7,7,7,7],[8,8,8,8]])
z=np.array([[9,9,9,9], [10,10,10,10],[12,12,26,26],[13,13,25,25]])

K = np.zeros([6,6])  
e=[x,y,z]
MC=[[0,1],[1,2],[2,0]]
for i in range(len(MC)): 
    K0=copy.copy(K)
    n1=MC[i][0]*2
    n2=MC[i][1]*2
    if i==0:
        kLoc=x
    elif i==1:
        kLoc=y
    else:
        kLoc=z
    K[n1:n1+2,n1:n1+2]+=kLoc[0:2,0:2]
    K[n1:n1+2,n2:n2+2]+=kLoc[0:2,2:4]
    K[n2:n2+2,n1:n1+2]+=kLoc[2:4,0:2]
    K[n2:n2+2,n2:n2+2]+=kLoc[2:4,2:4]
    print(K-K0)